# BLINK Tutorial

In [1]:
import sys
sys.path.insert(0, '../')

import blink
import pandas as pd
from ftplib import FTP

## Step 0: Load Input Data From GNPS

#### GNPS-LIBRARY Downloaded Here:
https://gnps-external.ucsd.edu/gnpslibrary

In [2]:
gnps_host = 'massive.ucsd.edu'
gnps_user = 'MSV000083475'
gnps_passwd = 'a'

In [3]:
filename = '7H12_9_1_song-75-s004-a04.mzML'    

with FTP(gnps_host, gnps_user, gnps_passwd) as ftp:
    ftp.cwd('/ccms_peak/RAW/PLATE7/')
    with open(filename, 'wb') as f:
        ftp.retrbinary('RETR ' + filename, f.write)

## Step 1: Read Input Data

#### BLINK can accept MGF files, mzML files, and lists of spectra as input. Correctly formatted lists of spectra and precursor m/zs can be fed directly into the discretization function, the other file formats can be parsed using a built-in helper function. 

__blink.open_msms_file():__
<br>
This function takes an MGF file or mzML file and converts into a Pandas DataFrame.

__Example Code:__

In [4]:
#parsing MGF:
mgf = blink.open_msms_file('GNPS-LIBRARY.mgf')

#parsing mzML:
mzml = blink.open_msms_file('7H12_9_1_song-75-s004-a04.mzML')

In [5]:
mgf.head()

,pepmass,charge,mslevel,source_instrument,filename,seq,ionmode,organism,name,pi,...,smiles,inchi,inchiaux,pubmed,submituser,libraryquality,spectrumid,scans,spectrum,precursor_mz
0,"(981.54, None)",[0+],2,LC-ESI-qTof,130618_Ger_Jenia_WT-3-Des-MCLR_MH981.4-qb.1.1....,*..*,Positive,GNPS-LIBRARY,3-Des-Microcystein_LR M+H,Gerwick,...,CC(C)CC1NC(=O)C(C)NC(=O)C(=C)N(C)C(=O)CCC(NC(=...,N/A,N/A,N/A,mwang87,1,CCMSLIB00000001547,1,"[[289.286377, 295.545288, 298.489624, 317.3249...",981.540
1,"(940.25, None)",[0+],2,LC-ESI-qTof,20111105_Anada_Ger_HoiamideB_MH940_qb.1.1..mgf,*..*,Positive,GNPS-LIBRARY,Hoiamide B M+H,Gerwick,...,CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](...,InChI=1S/C45H73N5O10S3/c1-14-17-24(6)34(52)26(...,N/A,N/A,mwang87,1,CCMSLIB00000001548,1,"[[278.049927, 278.957642, 281.258667, 291.9960...",940.250
2,"(456.1, None)",[0+],2,LC-ESI-qTof,20111105_Jenia_Ger_MalyngamideC_MH_456_qb.1.1....,*..*,Positive,GNPS-LIBRARY,Malyngamide C M+H,Gerwick,...,CCCCCCC[C@@H](C/C=C/CCC(=O)NC/C(=C/Cl)/[C@@]12...,InChI=1S/C24H38ClNO5/c1-3-4-5-6-8-11-19(30-2)1...,N/A,N/A,mwang87,1,CCMSLIB00000001549,1,"[[128.838745, 132.075684, 132.830322, 134.0384...",456.100
3,"(545.0, None)",[0+],2,LC-ESI-Ion Trap,20111105_Jenia_Ger_Scytonemin_MH_545_qb.1.1..mgf,*..*,Positive,GNPS-LIBRARY,Scytonemin M+H,Gerwick,...,OC1=CC=C(\C=C2\C(=O)C(C3=C4C5=C(C=CC=C5)N=C4\C...,InChI=1S/C36H20N2O4/c39-21-13-9-19(10-14-21)17...,CGZKSPLDUIRCIO-RPCRKUJJSA-N,N/A,mwang87,1,CCMSLIB00000001550,1,"[[343.896484, 345.458496, 372.684021, 386.0974...",545.000
4,"(314.116, None)",[1+],2,LC-ESI-qTof,A1.mgf,*..*,Positive,GNPS-LIBRARY,Salinisporamide A M+H,Fenical-Jensen-Moore,...,N/A,N/A,N/A,N/A,mwang87,1,CCMSLIB00000001551,1,"[[101.015465, 101.059807, 103.002441, 103.0285...",314.116


In [6]:
mzml.head()

,id,ms_level,rt,spectrum,precursor_mz,i,charge
0,29,2,0.074056,"[[51.023495, 52.39509, 53.00283, 53.039024, 56...",149.059667,2.787562e+06,1.0
1,30,2,0.077625,"[[57.16054, 62.25457, 89.9055, 109.89806, 111....",337.104489,1.578322e+06,1.0
2,31,2,0.081175,"[[53.03918, 54.165733, 64.39659, 77.03844, 82....",163.075234,7.590549e+05,1.0
3,32,2,0.084741,"[[50.71475, 54.151115, 66.84381, 67.95492, 78....",360.180406,7.438408e+05,1.0
4,34,2,0.094643,"[[64.33035, 84.11954, 90.76483, 103.29681, 110...",388.211635,3.552219e+05,1.0


## Step 2: "Discretize" Spectra

#### Prior to scoring, each set of spectra is discretized. This process converts lists of m/z and intensity arrays and precursor m/zs into dictionary-based sparse matrices.  

__blink.discretize_spectra():__
<br>
This function takes lists of fragmentation spectra and precursor m/zs as input and outputs a dictionary that contains intensity values, binned m/z values, and precursor m/z values. 
<br>
__Parameters:__
<br>
1. bin_width (default=0.001). This value is used to convert the m/z floats into rounded integer bins. Using smaller bin widths, precision increases and speed decreases. Example of binning calculation using default values. 100.002 --> 100002
2. intensity_power (default=0.5). The intensity power parameter scales intensity values used in the scoring. 
3. trim_empty (default=False). If True, spectra and associated metadata will be removed if the spectra are empty.
4. remove_duplicates (default=False). This parameter optionally calls blink.remove_duplicate_ions, which averages m/z values and sums the intensities of fragment ions in a spectrum that are within a minumum distance. This can be useful for spectra that are noisy or poorly centroided.
5. calc_network_score (default=True). Optionally, BLINK can be used for molecular networking. If enabled in this step, the function will calculate another set of spectra where the m/z are subtracted from the precursor m/z for that particular spectrum. 
6. metadata (default=None). Any desired metadata is stored with this variable. If the default of None is used, the metadata will be the number of ions in each spectrum. 

__Example Code:__

In [7]:
discretized_mgf = blink.discretize_spectra(mgf.spectrum.tolist(), mgf.precursor_mz.tolist(),
                                         bin_width=0.001, intensity_power=0.5, trim_empty=False, remove_duplicates=False, calc_network_score=False,
                                         metadata=mgf.drop(columns=['spectrum']).to_dict(orient='records'))

discretized_mzml = blink.discretize_spectra(mzml.spectrum.tolist(), mzml.precursor_mz.tolist(),
                                         bin_width=0.001, intensity_power=0.5, trim_empty=False, remove_duplicates=False, calc_network_score=False,
                                         metadata=mzml.drop(columns=['spectrum']).to_dict(orient='records'))

../blink.py:102: RuntimeWarning: divide by zero encountered in double_scalars
  inorm = np.array([1./np.linalg.norm(mzi[1]**intensity_power) for mzi in mzis])


Discretized spectra can be pre-computed and saved as numpy npz files using __blink.write_sparse_msms_file()__.

In [8]:
blink.write_sparse_msms_file('sparse_mgf.npz', discretized_mgf)
blink.write_sparse_msms_file('sparse_mzml.npz', discretized_mzml)

Likewise, saved pre-computed sparse spectra can be read with __blink.open_sparse_msms_file()__.

In [9]:
sparse_mgf = blink.open_sparse_msms_file('sparse_mgf.npz')
sparse_mzml = blink.open_sparse_msms_file('sparse_mzml.npz')

## Step 3: Score Spectra

#### Next, the two sets of spectra are scored against eachother. Given discretized spectra inputs, a matrix of pairwise scores is generated.

__blink.score_sprase_spectra():__
<br>
This function calculates the pairwise score and matching ion count matrices.
<br>
__Parameters:__
1. tolerance (default=0.01). The tolerance parameter is the minumum value between fragment ion m/zs to be considered "matching" and factor into the cosine based score. However, the true tolerance of the scoring algorithm is determined by both the "tolerance" parameter and the bin width such that true tolerance is "tolerance" - "bin_width".
2. mass_diffs (default=[0]). This optional parameter allows the user to score spectra against spectra shifted by user defined chemical masses. The default value of 0 does not shift the spectra. This feature is still in development. 
3. react_steps (default=1). This value expands the mass_diffs by a specified number of steps. This feature is still in development
4. calc_network_score (default=True). Whether or not to calculate the molecular network score. If True, the nuetral loss spectra computed during disrectization of scored. 

In [10]:
S12 = blink.score_sparse_spectra(sparse_mgf, sparse_mzml, tolerance=0.01, mass_diffs=[0], react_steps=1, calc_network_score=False)

In [11]:
#cosine similarity scores
S12['mzi']

<12914x3038 sparse matrix of type '<class 'numpy.float64'>'
	with 17671043 stored elements in Compressed Sparse Row format>

In [12]:
#matching ion counts
S12['mzc']

<12914x3038 sparse matrix of type '<class 'numpy.int64'>'
	with 17671043 stored elements in Compressed Sparse Row format>

## Step 4: Filter Scores

#### BLINK has several helper functions to convert the raw score and count matrices into human readable outputs. 

__blink.filter_hits()__
<br>
This function filters the score and count matrices to only include those above a user defined cutoff
<br>
__Optional Parameters:__
1. good_score (default=0.5). This is the lowest score that will be kept from the score matrix.
2. min_matches (default=5). The minumum number of matching ions to keep the score. 
3. good_matches (default=20). Keep scores with greater than or equal to this number of matches even if the score is lower than the good_score variable.
4. calc_network_score (default=True). If true, filter scores using the maximum of the precursor shifted and un-shifted score/matches. 

In [13]:
filtered_S12 = blink.filter_hits(S12, min_matches=5, good_matches=20, good_score=0.6, calc_network_score=False)

__blink.create_blink_matrix():__
<br>
This is a helper function that reshapes the score/matches matrices such that they can be easily associated with the spectral metadata
<br>
__Optional Parameters:__
1. calc_network_score (default=True). If true, create reshaped array with network score/matches as data

In [14]:
m = blink.create_blink_matrix_format(filtered_S12, calc_network_score=False)

In [15]:
df = pd.DataFrame(m, columns=['raveled_index', 'query', 'ref', 'score', 'matches'])
df = pd.merge(df, pd.DataFrame(S12['S1_metadata']).add_suffix('_query'), left_on='query', right_index=True, how='left')
df = pd.merge(df, pd.DataFrame(list(S12['S2_metadata'])).add_suffix('_ref'), left_on='ref', right_index=True, how='left')

In [16]:
df.head()

,raveled_index,query,ref,score,matches,0_query,id_ref,ms_level_ref,rt_ref,precursor_mz_ref,i_ref,charge_ref,num_ions_ref
0,12240.0,4.0,88.0,0.115395,26.0,"{'pepmass': (314.116, None), 'charge': [1+], '...",170,2,0.511913,310.128258,6.478345e+06,1.0,105
1,12250.0,4.0,98.0,0.148134,21.0,"{'pepmass': (314.116, None), 'charge': [1+], '...",183,2,0.556523,280.117800,8.729133e+05,NaN,69
2,12557.0,4.0,405.0,0.086961,20.0,"{'pepmass': (314.116, None), 'charge': [1+], '...",566,2,1.859129,265.155100,2.832238e+06,NaN,108
3,12712.0,4.0,560.0,0.097227,20.0,"{'pepmass': (314.116, None), 'charge': [1+], '...",760,2,2.507878,211.075390,2.571002e+06,1.0,96
4,12724.0,4.0,572.0,0.080610,25.0,"{'pepmass': (314.116, None), 'charge': [1+], '...",775,2,2.559793,265.143100,2.030908e+06,NaN,136


## Using Task Runner

#### BLINK also includes a simple task-running function that covers many use cases.

__blink.get_blink_hits()__
<br>
This function takes input data as Pandas DataFrames that contain a "spectrum" and "precursor_mz" column, or mzML/MGF data files. Output is a filtered, formatted DataFrame with scores and matches between all MS2 spectra.
<br>
__Optional Parameters:__
1. calc_network_score (default=True): This parameter determines whether or not the network score (max of precursor mz shifted and unshifted mz score/matches) is calculated.
2. min_matches (default=5): See blink.filter_hits
3. good_matches (default=20): See blink.filter_hits
4. good_score (default=0.55): See blink.filter_hits
5. precursor_match (default=5): If not False, the output DataFrame is filtered to remove comparisons with precursor m/zs with a greater difference than the value of precursor_match in ppm.

In [17]:
df = blink.get_blink_hits(mgf, mzml, calc_network_score=False, precursor_match=5)

../blink.py:102: RuntimeWarning: divide by zero encountered in double_scalars
  inorm = np.array([1./np.linalg.norm(mzi[1]**intensity_power) for mzi in mzis])


In [18]:
df.head()

,raveled_index,query,ref,score,matches,pepmass_query,charge_query,mslevel_query,source_instrument_query,filename_query,...,precursor_mz_ref,i_ref,charge_ref,num_ions_ref,precursor_ppm_diff,jaccard_matches,overlap_matches,score_rank,matches_rank,jaccard_matches_rank
1458,207514.0,68.0,930.0,0.414345,35.0,"(323.258, None)",[1+],2,LC-ESI-qTof,C9.mgf,...,323.258236,970822.125,1.0,107,0.730567,0.230263,0.4375,2.0,4.0,2.0
1689,207876.0,68.0,1292.0,0.469656,46.0,"(323.258, None)",[1+],2,LC-ESI-qTof,C9.mgf,...,323.258019,6095855.000,1.0,174,0.058503,0.221154,0.5750,1.0,1.0,4.0
1852,208350.0,68.0,1766.0,0.388938,41.0,"(323.258, None)",[1+],2,LC-ESI-qTof,C9.mgf,...,323.257627,5735303.000,1.0,139,1.155004,0.230337,0.5125,3.0,2.0,1.0
1883,208530.0,68.0,1946.0,0.353129,31.0,"(323.258, None)",[1+],2,LC-ESI-qTof,C9.mgf,...,323.257800,1955164.750,NaN,103,0.618701,0.203947,0.3875,5.0,5.0,5.0
1906,208626.0,68.0,2042.0,0.367442,36.0,"(323.258, None)",[1+],2,LC-ESI-qTof,C9.mgf,...,323.258327,1891701.125,1.0,118,1.012563,0.222222,0.4500,4.0,3.0,3.0
